INICIAR COM BIBLIOTECAS E ARQUIVOS DE ENTRADA

In [33]:
# IMPORTACAO BIBLIOTECAS PRINCIPAIS
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import itertools

# CONECTORES DO SNOWFLAKE
import swifter
import re
import os
import glob
import datetime
import snowflake.connector
import time
import warnings

In [35]:
# CSV EXTRAIDOS DO PLSQL
tabelas_csv_path = 'arquivos_entrada/campos_tabelas copy.csv'
volume_csv_path = 'arquivos_entrada/volume.csv'

# LEITURA DOS ARQUIVOS EM CSV
tabela_chaves = pd.read_csv(tabelas_csv_path, sep=';', encoding='utf-8')
tabela_volumes = pd.read_csv(volume_csv_path, sep=';', encoding='utf-8')

MANIPULACOES COM APLICACAO DE CLUSTER E VOLUMETRIA

In [36]:
# CATEGORIZAR VOLUME EM FAIXAS
def faixa_volume(num_rows):
    if num_rows <= 10000:
        return 'BAIXO'
    elif num_rows <= 100000:
        return 'MODERADO'
    elif num_rows <= 1000000:
        return 'MEDIO'
    else:
        return 'ALTO'

tabela_volumes['VOLUMETRIA'] = tabela_volumes['NUM_ROWS'].apply(faixa_volume)

tabela_volumes.head(10)

,SCHEMA,TABLE_NAME,NUM_ROWS,VOLUMETRIA
0,DELTA,SEG_PRODUTOGRUPOSCRIPTSEG,4390,BAIXO
1,DELTA,SEG_DADOSOCORRENCIA,9911519,ALTO
2,DELTA,SEG_ATENDIMENTOWEB,5902582,ALTO
3,DELTA,SEG_APOLICEITEM,174198060,ALTO
4,DELTA,SEG_ATENDIMENTO,17134456,ALTO
5,DELTA,SEG_APOLICEITEMAUXILIAR,160022807,ALTO
6,DELTA,SEG_PRODUTOGRUPOSCRIPT,511,BAIXO
7,DELTA,SEG_APOLICECONTRATO,201303718,ALTO
8,DELTA,GEN_PRODUTOGRUPOSCRIPT,517,BAIXO
9,DELTA,GEN_GRUPOVEICULO,1762,BAIXO


In [37]:
# Função para calcular a similaridade entre nomes de tabelas
def comparar_nomes_tabelas(tabela_1, tabela_2):
    similaridade = fuzz.token_set_ratio(tabela_1, tabela_2)
    return similaridade

def chaves_em_comum(tabela_chaves, tabela_1, tabela_2):
    chaves1 = set(tabela_chaves.query('TABLE_NAME == @nome_tabela_1 and TABLE_KEY == "SIM"')['COLUMN_NAME'].unique())
    chaves2 = set(tabela_chaves.query('TABLE_NAME == @nome_tabela_2 and TABLE_KEY == "SIM"')['COLUMN_NAME'].unique())
    chaves_comum = chaves1.intersection(chaves2)
    return list(chaves_comum)

In [46]:
resultados = []
comparacoes_realizadas = set()  # ARMAZENAR PARES JÁ COMPARADOS
tabelas_nomes = list(tabela_chaves['TABLE_NAME'].unique())

for i in range(len(tabelas_nomes)):
    tabelas_nomes_comparaveis = tabelas_nomes.copy()
    tabelas_nomes_comparaveis.remove(tabelas_nomes[i])

    for j in range(len(tabelas_nomes_comparaveis)):
        nome_tabela_1 = tabelas_nomes[i]
        nome_tabela_2 = tabelas_nomes_comparaveis[j]

        # CRIAR PAR ORDENADO PARA NÃO DUPLICAR
        par = tuple(sorted([nome_tabela_1, nome_tabela_2]))
        
        if par not in comparacoes_realizadas:
            similaridade = comparar_nomes_tabelas(
                tabela_chaves.query('TABLE_NAME == @nome_tabela_1 and TABLE_KEY == "SIM"'),
                tabela_chaves.query('TABLE_NAME == @nome_tabela_2 and TABLE_KEY == "SIM"')
            )

            threshold = 70

            if similaridade >= threshold:
                resultados.append({
                    'TABLE_NAME1': nome_tabela_1,
                    'TABLE_NAME2': nome_tabela_2,
                    'TYPE_SIMILARITY': f'MODEL >= {threshold}'
                })
            else:

                chaves_1 = chaves_em_comum(tabela_chaves, nome_tabela_1, nome_tabela_2)
                if len(chaves_1) >= 3:
                    resultados.append({
                        'TABLE_NAME1': nome_tabela_1,
                        'TABLE_NAME2': nome_tabela_2,
                        'TYPE_SIMILARITY': f'KEYS'
                    })
                else:
                    resultados.append({
                        'TABLE_NAME1': nome_tabela_1,
                        'TABLE_NAME2': nome_tabela_2,
                        'TYPE_SIMILARITY': f'NONE'
                    })

            comparacoes_realizadas.add(par)  # Adicionando o par ao conjunto
            
df_resultados = pd.DataFrame(resultados)

# Exibindo o DataFrame
df_resultados.head(10)

,TABLE_NAME1,TABLE_NAME2,TYPE_SIMILARITY
0,SEG_ATENDIMENTO,SEG_ATENDIMENTOABA,NONE
1,SEG_ATENDIMENTO,SEG_ATENDIMENTOANALISE,NONE
2,SEG_ATENDIMENTO,SEG_ATENDIMENTOANALISEIMAGEM,NONE
3,SEG_ATENDIMENTO,SEG_ATENDIMENTOANALISEIMAGEMEX,NONE
4,SEG_ATENDIMENTO,SEG_ATENDIMENTOAPOLICECANC,NONE
5,SEG_ATENDIMENTO,SEG_ATENDIMENTOARQUIVO,NONE
6,SEG_ATENDIMENTO,SEG_ATENDIMENTOATRIBUTO,NONE
7,SEG_ATENDIMENTO,SEG_ATENDIMENTOAVALIACAODANO,NONE
8,SEG_ATENDIMENTO,SEG_ATENDIMENTOBLOQUEIO,MODEL >= 70
9,SEG_ATENDIMENTO,SEG_ATENDIMENTOCANAL,NONE


In [ ]:
# IDENTIFICAR E ASSOCIAR CLUSTER A TABELA DE VOLUME
def identificar_clusters(tabela, tabela_clusters):
    clusters_encontrados = []
    
    for _, cluster_row in tabela_clusters.iterrows():
        chave = cluster_row['CHAVE']
        cluster = cluster_row['CLUSTER']
        
        if chave in tabela:
            clusters_encontrados.append(cluster)
    
    if clusters_encontrados:
        return ','.join(clusters_encontrados)
    return 'NAO IDENTIFICADO'

def associar_clusters(tabela_volumes, tabelas_clusters):
    tabela_volumes['CLUSTERS'] = tabela_volumes['TABLE_NAME'].apply(lambda tabela: identificar_clusters(tabela, tabela_clusters))
    return tabela_volumes

tabela_volume_complementada = associar_clusters(tabela_volumes, tabela_clusters)

df_padronizar = pd.DataFrame(tabela_volume_complementada, columns=['SCHEMA', 'CLUSTERS', 'TABLE_NAME', 'VOLUMETRIA', 'NUM_ROWS'])
df_padronizar['CLUSTERS'] = df_padronizar['CLUSTERS'].str.split(',')
df_padronizar = df_padronizar.explode('CLUSTERS').reset_index(drop=True)

tabela_volume_complementada = df_padronizar

tabela_volume_complementada.head(10)

In [42]:
# COMBINAR RESULTADOS COM VOLUMETRIA

FAZER JOIN COM OS DOIS CAMPOS DE TABELA PARA DESCOBRIR VOLUMETRIA

KeyError: 'Tabela 1'

In [5]:
# COMBINAR TABELA DE CHAVES COM VOLUME E CLUSTERS
def complementar_tabela_chaves(tabela_chaves, tabela_volume):
    return pd.merge(tabela_chaves, tabela_volume[['CLUSTERS', 'VOLUMETRIA', 'TABLE_NAME', 'SCHEMA']], how='left', on=['TABLE_NAME', 'SCHEMA'])

df_completa = complementar_tabela_chaves(tabela_chaves, tabela_volume_complementada)

df_completa = df_completa.dropna(subset=['CONSTRAINT_TYPE'])

df_completa

,SCHEMA,TABLE_NAME,COLUMN_NAME,CONSTRAINT_TYPE,CLUSTERS,VOLUMETRIA
2,DELTA,GEN_PRODUTOGRUPOSCRIPT,CODGRUPO,FK,PRODUTO,BAIXO
3,DELTA,GEN_PRODUTOGRUPOSCRIPT,CODGRUPO,FK,SCRIPT,BAIXO
4,DELTA,GEN_PRODUTOGRUPOSCRIPT,CODGRUPO,PK,PRODUTO,BAIXO
5,DELTA,GEN_PRODUTOGRUPOSCRIPT,CODGRUPO,PK,SCRIPT,BAIXO
6,DELTA,GEN_PRODUTOGRUPOSCRIPT,CODSCRIPT,FK,PRODUTO,BAIXO
...,...,...,...,...,...,...
1250,DELTA,SEG_SEGURADORAVEICULO,CODSEGURADORA,PK,SEGURADORA,ALTO
1253,DELTA,SEG_SEGURADORAVEICULO,CODSEGURADORA,FK,VEICULO,ALTO
1254,DELTA,SEG_SEGURADORAVEICULO,CODSEGURADORA,FK,SEGURADORA,ALTO
1255,DELTA,SEG_SEGURADORAVEICULO,CODUSUARIOMAPEAMENTO,FK,VEICULO,ALTO


In [6]:
def tabelas_comparaveis(df_completa):
    resultados = []

    # ITERA SOBRE TABELAS UNICAS
    for row1, row2 in itertools.combinations(df_completa.itertuples(index=False), 2):
        
        # VALIDAR SCHEMA, CAMPOS, TABELAS E VOLUMETRIA
        if (row1.SCHEMA == row2.SCHEMA and
            row1.TABLE_NAME != row2.TABLE_NAME and
            row1.COLUMN_NAME == row2.COLUMN_NAME and
            row1.CLUSTERS == row2.CLUSTERS):

            if row1.VOLUMETRIA == row2.VOLUMETRIA:
                volumetria = row1.VOLUMETRIA
            else:
                volumetria = 'DIFERENTE'

            resultados.append({
                'CLUSTERS': row1.CLUSTERS,
                'SCHEMA': row1.SCHEMA,
                'VOLUMETRIA': volumetria,
                'TABLE_NAME1': row1.TABLE_NAME,
                'TABLE_NAME2': row2.TABLE_NAME
            })

    # TABELAS não IDENTIFICADAS
    tabelas_identificadas = set(itertools.chain(*[(res['TABLE_NAME1'], res['TABLE_NAME2']) for res in resultados]))
    
    for row in df_completa.itertuples(index=False):
        if row.TABLE_NAME not in tabelas_identificadas:
            resultados.append({
                'CLUSTERS': row.CLUSTERS,
                'SCHEMA': row.SCHEMA,
                'VOLUMETRIA': row.VOLUMETRIA,
                'TABLE_NAME1': row.TABLE_NAME,
                'TABLE_NAME2': 'NAO IDENTIFICADO'
            })

    # CRIAR DATAFRAME COM RESULTADOS
    df_tabelas_associadas = pd.DataFrame(resultados)

    # REMOVER DUPLICADAS E PARES INVERTIDOS
    df_tabelas_associadas['par'] = df_tabelas_associadas.apply(lambda x: tuple(sorted([x['TABLE_NAME1'], x['TABLE_NAME2']])), axis=1)
    df_unique_pairs = df_tabelas_associadas.drop_duplicates(subset='par').drop(columns='par').reset_index(drop=True)

    return df_unique_pairs

# ASSOCIAR TABELAS COMPARADAS
df_tabelas_associadas = tabelas_comparaveis(df_completa)

df_tabelas_associadas.head(10)


,CLUSTERS,SCHEMA,VOLUMETRIA,TABLE_NAME1,TABLE_NAME2
0,PRODUTO,DELTA,BAIXO,GEN_PRODUTOGRUPOSCRIPT,SEG_PRODUTOGRUPOSCRIPT
1,PRODUTO,DELTA,BAIXO,GEN_PRODUTOGRUPOSCRIPT,SEG_PRODUTOGRUPOSCRIPTSEG
2,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATO
3,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOANEXO
4,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOPLANO
5,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOSERVICO
6,APOLICE,DELTA,ALTO,SEG_APOLICECONTRATO,SEG_APOLICEITEM
7,APOLICE,DELTA,ALTO,SEG_APOLICECONTRATO,SEG_APOLICEITEMAUXILIAR
8,APOLICE,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_APOLICESEGURADORA
9,APOLICE,DELTA,ALTO,SEG_APOLICEITEM,SEG_APOLICEITEMAUXILIAR


CALCULAR SIMILARIDADE POR MODELO

In [7]:
# CALCULAR A SIMILARIDADE POR ÍNDICE DE JACCARD
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0

def calcular_similaridade_jaccard(df_tabelas_associadas, tabela_chaves):
    similaridades = []

    # ITERAR SOBRE LINHAS df_tabelas_associadas
    for _, row in df_tabelas_associadas.iterrows():
  
        colunas_TABLE_NAME1 = set(tabela_chaves[tabela_chaves['TABLE_NAME'] == row['TABLE_NAME1']]['COLUMN_NAME'])
        colunas_TABLE_NAME2 = set(tabela_chaves[tabela_chaves['TABLE_NAME'] == row['TABLE_NAME2']]['COLUMN_NAME'])
        
        # Calcula a similaridade de Jaccard, garantindo que ambas as tabelas têm colunas
        similaridade_jaccard = jaccard_similarity(colunas_TABLE_NAME1, colunas_TABLE_NAME2)
        similaridades.append(similaridade_jaccard)
    
    # CRIA UMA COLUNA PARA INFORMAR A SIMILARIDADE
    df_tabelas_associadas['SIMILARIDADE_JACCARD'] = similaridades
    
    return df_tabelas_associadas 

# FUNÇÃO PARA CALCULAR SIMILARIDADE
df_similaridade_jaccard = calcular_similaridade_jaccard(df_tabelas_associadas, tabela_chaves)

df_similaridade_jaccard.head(10)

,CLUSTERS,SCHEMA,VOLUMETRIA,TABLE_NAME1,TABLE_NAME2,SIMILARIDADE_JACCARD
0,PRODUTO,DELTA,BAIXO,GEN_PRODUTOGRUPOSCRIPT,SEG_PRODUTOGRUPOSCRIPT,1.000000
1,PRODUTO,DELTA,BAIXO,GEN_PRODUTOGRUPOSCRIPT,SEG_PRODUTOGRUPOSCRIPTSEG,1.000000
2,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATO,0.093750
3,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOANEXO,0.300000
4,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOPLANO,0.214286
5,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOSERVICO,0.176471
6,APOLICE,DELTA,ALTO,SEG_APOLICECONTRATO,SEG_APOLICEITEM,0.068493
7,APOLICE,DELTA,ALTO,SEG_APOLICECONTRATO,SEG_APOLICEITEMAUXILIAR,0.014151
8,APOLICE,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_APOLICESEGURADORA,0.250000
9,APOLICE,DELTA,ALTO,SEG_APOLICEITEM,SEG_APOLICEITEMAUXILIAR,0.033835


CONECTAR COM SNOWFLAKE PARA VALIDAR EXISTENCIA DE TABELAS

In [9]:
# INICIAR CURSOR COM SNOWFLAKE
con = snowflake.connector.connect(
    user="WELLINGTON.MELO@AUTOGLASS.COM.BR", 
    account="ZGVSIFS-CBB87909",
    role='GL_SNOWFLAKE_ACESSO_MAX_ENG_ANALITICO',    
    warehouse="WH_USE_MAXPAR",
    authenticator="externalbrowser")
cx = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/00466ab9-c560-49f0-86af-238be4a1ac67/saml2?SAMLRequest=lZJRb5swFIX%2FCvKeAUMoAytJRRtFy5ptWUJbqW8OmMStsZmvCW1%2B%2FQxJpO6hlfaGzDn%2Bju%2B54%2BvXWjgHpoErOUGBh5HDZKFKLncTdJ%2FP3QQ5YKgsqVCSTdAbA3Q9HQOtRUOy1uzlmv1pGRjHXiSB9D8mqNWSKAociKQ1A2IKssl%2BLEnoYUIBmDYWh86WErhl7Y1piO93Xed1I0%2FpnR9ijH2c%2BlbVS76gd4jmc0ajlVGFEhfLq33TB4jAx1GPsApLWJ2NN1yeRvAZZXsSAfmW5yt39WuTIye7vO5WSWhrpjdMH3jB7tfLUwCwCfbHbRglaeK14DIKxg08kKqrBH1hhaqb1thrPfvlV6z0hdpxO6zFbIKaF16uqyyX5knAqnnIb%2BBOzo%2BP9Pfd9%2BckK5%2BvonXE0%2BUhzh7huCuQ83CpNuyrXQC0bCH7Qo09wmHkBoGL4xxjMgpIGHpJip%2BQM7OFcknN4LykHnJ4NS%2B0AlUZJQWXbEiJcRTHdJu6xVWM3SitsJvEtHLDUbJlEQ1oEX%2F1%2B9pCdFodMgTR0%2F8eyNh%2Fbz%2Bv4U%2FbzGK2UoIXb85c6Zqaj4sLvGA44aVbDVLCaspFVpaaAdgChVDdrWbU2G03

In [10]:
# TRAZER TODAS AS TABELAS
cx.execute("USE SECONDARY ROLES ALL")
cx.execute("USE ROLE GL_SNOWFLAKE_ACESSO_MAX_ENG_ANALITICO")
cx.execute("USE WAREHOUSE WH_USE_MAXPAR")

In [14]:
# BUSCAR TABELAS NOS DIFERENTES DATABASES
query = """

    WITH TABELAS AS (

        SELECT * FROM SANDBOX.INFORMATION_SCHEMA.TABLES

        WHERE TABLE_SCHEMA != 'INFORMATION_SCHEMA' AND TABLE_NAME NOT LIKE '%TRU_%'

        UNION ALL
        
        SELECT * FROM MAXPAR.INFORMATION_SCHEMA.TABLES

        WHERE TABLE_SCHEMA != 'INFORMATION_SCHEMA' AND TABLE_NAME NOT LIKE '%TRU_%'

    )

    SELECT DISTINCT TABLE_NAME AS TABELAS

    FROM TABELAS

"""

In [15]:
# ARMAZENAR DADOS NA TABELA
tabelas_incluidas = pd.read_sql(query, con)

tabelas_incluidas

C:\Users\wellington.melo\AppData\Local\Temp\ipykernel_14032\2977033788.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabelas_incluidas = pd.read_sql(query, con)


,TABELAS
0,SEG_CONTATOATENDIMENTO
1,GEN_FORNECEDOR
2,SEG_ATENDIMENTO
3,GEN_PRODUTOPARTNUMBER
4,SEG_TIPOCONSULTAMAPEAMENTO
...,...
151,SEG_PRODUTOGRUPO
152,GEN_GRUPOCLIENTE
153,SEG_ATENDIMENTOCOMPRADOR
154,SEG_ITEMPRODUTO


In [19]:
# COMPARAR TABELAS PARA AVALIAR EXISTENCIA NO SNOWFLAKE
tabela_consolidada = df_similaridade_jaccard['EXISTE_SNOWFLAKE'] = df_similaridade_jaccard['TABLE_NAME2'].isin(tabelas_incluidas['TABELAS']).map({True: 'SIM', False: 'NÃO'})

tabela_consolidada = df_similaridade_jaccard
tabela_consolidada.head(10)

,CLUSTERS,SCHEMA,VOLUMETRIA,TABLE_NAME1,TABLE_NAME2,SIMILARIDADE_JACCARD,EXISTE_SNOWFLAKE
0,PRODUTO,DELTA,BAIXO,GEN_PRODUTOGRUPOSCRIPT,SEG_PRODUTOGRUPOSCRIPT,1.000000,NÃO
1,PRODUTO,DELTA,BAIXO,GEN_PRODUTOGRUPOSCRIPT,SEG_PRODUTOGRUPOSCRIPTSEG,1.000000,NÃO
2,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATO,0.093750,SIM
3,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOANEXO,0.300000,SIM
4,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOPLANO,0.214286,NÃO
5,CONTRATO,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_CONTRATOSERVICO,0.176471,NÃO
6,APOLICE,DELTA,ALTO,SEG_APOLICECONTRATO,SEG_APOLICEITEM,0.068493,SIM
7,APOLICE,DELTA,ALTO,SEG_APOLICECONTRATO,SEG_APOLICEITEMAUXILIAR,0.014151,SIM
8,APOLICE,DELTA,DIFERENTE,SEG_APOLICECONTRATO,SEG_APOLICESEGURADORA,0.250000,NÃO
9,APOLICE,DELTA,ALTO,SEG_APOLICEITEM,SEG_APOLICEITEMAUXILIAR,0.033835,SIM


EXPORTAR RESULTADOS

In [10]:
# EXPORTAR O CSV
tabela_consolidada.to_csv('resultado.csv', index=False)